In [1]:
#Importing other jupyter notebook (calculations)
import Pkg
Pkg.add("NBInclude")
Pkg.add("IJulia")
using NBInclude, IJulia

@nbinclude("plumeSSMEJP.ipynb")
IJulia.installkernel("Julia nodeps", "--depwarn=no")

   Updating registry at `~/.julia/registries/General`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Pr

started solver :)


┌ Warning: Assignment to `χ_1` in soft scope is ambiguous because a global variable by the same name exists: `χ_1` will be treated as a new local. Disambiguate by using `local χ_1` to suppress this warning or `global χ_1` to assign to the existing global variable.
└ @ nothing /home/chinahg/GCresearch/rocketemissions/plumeSSMEJP.ipynb:In[20]:41


Splitting complete


In [3]:
#PLOTTING ONLY IN THIS NOTEBOOK
#REGRID SOLUTION
xx, yy, u_g, T_g, χ_gH2O =  regrid_solution(x, y, u, T, χ[:,:,6], 0.01);
xx, yy, u_g, T_g, χ_gNO2 =  regrid_solution(x,y, u, T, χ[:,:,37], 0.01);
xx, yy, u_g, T_g, χ_gN2O =  regrid_solution(x, y, u, T, χ[:,:,38], 0.01);
xx, yy, u_g, T_g, χ_gNO =  regrid_solution(x, y, u, T, χ[:,:,36], 0.01);

#PLOT 2D MAPS AND SAVE
plot_H2O = plot_heatmap(xx, yy, χ_gH2O, "Axial distance, m", "Radial distance, m","Concentration", :magma)
savefig("/home/chinahg/GCresearch/rocketemissions/rockettests/"*h_string*"m/X_H2O.png")

#CALCULATE NO EI FOR ALTITUDE
# Sum of all NOx species as a function of x
sumNO = zeros(n);
sumNO2 = zeros(n);
sumN2O = zeros(n);
i = 1
j = 1

for i in 1:n-1
    if xx[i] < 0 || xx[i] == NaN
        xx[i] = (xx[i-1] + xx[i+1])/2
    end
    for j in 1:50
        #println(j," ", i)
        #println(χ_gNO[j,i])
        if χ_gNO[j,i] < 0 || χ_gNO[j,i]== NaN 
            println(j)
            println(i)
            χ_gNO[j,i] = 0
        
        elseif χ_gNO2[j,i] < 0 || χ_gNO2[j,i]== NaN
            println(j)
            println(i)
            χ_gNO2[j,i] = 0
        
        elseif χ_gN2O[j,i] < 0 || χ_gN2O[j,i]== NaN
            println(j)
            println(i)
            χ_gN2O[j,i] = 0
        end
    end 
end
#print(χ_gNO[:,20])
i = 1
for i in 1:n
    sumNO[i] = sum(χ_gNO[:,i])
    sumNO2[i] = sum(χ_gNO2[:,i])
    sumN2O[i] = sum(χ_gN2O[:,i])
end
    
sumNOx = sumNO+sumNO2+sumN2O


┌ Warning: Duplicated knots were deduplicated. Use Interpolations.deduplicate_knots!(knots) explicitly to avoid this warning.
│   k1 = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
└ @ Interpolations /home/chinahg/.julia/packages/Interpolations/Glp9h/src/gridded/gridded.jl:77
┌ Warning: Duplicated knots were deduplicated. Use Interpolations.deduplicate_knots!(knots) explicitly to avoid this warning.
│   k1 = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
└ @ Interpolations /home/chinahg/.julia/packages/Interpolations/Glp9h/src/gridded/gridded.jl:77
┌ Warning: Duplicated knots were ded

100-element Array{Float64,1}:
   0.0
   0.00011253371222159729
   0.0001237718096906291
   0.00014821364068301387
   0.0002077680204595043
   0.0003757515579190791
   0.0009343429607220693
   0.003111231795940446
   0.012302661765649252
   0.04934048104234841
   0.18905299835493505
   0.7290452861220121
   2.7104458409655585
   ⋮
   0.0
  -3.290273369408192e-55
   0.0
  -1.3160450262145024e-54
   0.0
  -5.264438271592348e-54
   0.0
  -2.1057283671899017e-53
   0.0
  -8.422968369437254e-53
   0.0
 NaN